<ol>
<li> </li>
</ol>

In [1]:
from scipy.io import loadmat, savemat
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style
import tensorflow as tf
print('Tensorflow version', tf.__version__)

%matplotlib inline
matplotlib.style.use("ggplot")

Tensorflow version 1.1.0


In [166]:
synth_data = loadmat('./fabricated_mnist_data/data_and_labels.mat')
all_x = synth_data['data']
all_y = synth_data['labels']
print("X shape", all_x.shape)
print("Y 'shape'", all_y.shape)
print("Y first example shape", all_y[0, 0].shape)
print("Y last example shape", all_y[0, -1].shape)

X shape (23330, 28, 140)
Y 'shape' (1, 23330)
Y first example shape (1, 10)
Y last example shape (5, 10)


### Reshape y by padding < 5 length labels, and getting the length of the labels

In [167]:
all_y = all_y.reshape(-1)
all_y_lens = np.array([len(y_ex) for y_ex in all_y])
def pad(y_ex):
    toreturn = np.zeros((5, 10))
    toreturn[:len(y_ex), :] = y_ex
    return toreturn
pad(np.arange(10*2).reshape(2, 10))


array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [168]:
all_y = np.array([pad(y_ex) for y_ex in all_y])
all_y.shape

(23330, 5, 10)

In [171]:
def one_hot(num):
    num = np.ravel(num).astype(int)
    toreturn = np.zeros(shape=(len(num), 5), dtype=np.int16)
    for i, dig in enumerate(num):
        toreturn[i, dig-1] = 1
    return toreturn    

In [172]:
all_y_lens = one_hot(all_y_lens)

In [173]:
np.random.seed(999)
idx = np.arange(len(all_x))
np.random.shuffle(idx)
temp = len(all_x)//5

train_idx = idx[:(temp*3)]
valid_idx = idx[(temp*3):(temp*4)]
test_idx = idx[(temp*4):]

train_x = all_x[train_idx].reshape(-1, 28, 140, 1)
train_y = all_y[train_idx]
train_ylen = all_y_lens[train_idx]

valid_x = all_x[valid_idx].reshape(-1, 28, 140, 1)
valid_y = all_y[valid_idx]
valid_ylen = all_y_lens[valid_idx]

test_x = all_x[test_idx].reshape(-1, 28, 140, 1)
test_y = all_y[test_idx]
test_ylen = all_y_lens[test_idx]

In [174]:
def masked_xe(output, target):
    """Compute the average cross entropy of sequences up to the lengths of the targets.
    
    This assumes a target is of shape [N, L, K], where N is the number of examples, L is the maximum 
    sequence length, and K is the number of classes. Additionally, target should be zero-padded to
    arrive at the max sequence length for all sequences
    
    Borrowed from https://danijar.com/variable-sequence-lengths-in-tensorflow/.
    """
    cross_entropy = target * tf.log(output)
    cross_entropy = -tf.reduce_sum(cross_entropy, reduction_indices=2)
    mask = tf.sign(tf.reduce_max(tf.abs(target), reduction_indices=2))
    cross_entropy *= mask
    
    cross_entropy = tf.reduce_sum(cross_entropy, reduction_indices=1)
    cross_entropy /= tf.reduce_sum(mask, reduction_indices=1)
    return tf.reduce_mean(cross_entropy)

In [175]:
def get_pred(len_proba, dig_proba):
    length = np.argmax(len_proba, axis=-1)
    digits = np.argmax(dig_proba, axis=-1)
    
    return [''.join([str(x) for x in dig[:ilen+1]]) for ilen, dig in zip(length, digits)]
lens = [
    [0, 1]+[0]*3
]
digs = [
    [
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
    ]
]
get_pred(lens, digs)

['12']

In [176]:
lens = [
    [0, 1]+[0]*3,
    [0, 0, 0, 1]+[0]
]
digs = [
    [
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
    ],
    [
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
    ]
]

In [177]:
plens = [
    [0, 1]+[0]*3,
    [0, 0, 1]+[0]*2
]
pdigs = [
    [
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
    ],
    [
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
        [0, 1]+[0]*8,
        [0, 0.2, 0.8] + [0]*7,
    ]
]

In [178]:
get_pred(lens, digs)

['12', '1212']

In [179]:
get_pred(lens, digs)

['12', '1212']

In [180]:
def accuracy(y_true, y_len_true, y_pred, y_len_pred):
    actual = np.array(get_pred(y_len_true, y_true))
    pred = np.array(get_pred(y_len_pred, y_pred))
    return np.mean(actual == pred)
accuracy(digs, lens, pdigs, plens)

0.5

In [278]:
lr = 0.001
momentum = 0.7
len_balance = 0.6  # give the sequence length loss len_balance weight and the sequence digits loss (1-len_balance) weight 
simplest_graph =  tf.Graph()

In [279]:
with simplest_graph.as_default(): 
    tf_x = tf.placeholder(tf.float32, (None, all_x.shape[1], all_x.shape[2], 1))
    tf_y = tf.placeholder(tf.float32, (None, all_y.shape[1], all_y.shape[2]))
    tf_ylen = tf.placeholder(tf.int32, (None, 5))
    
    # (N, 28, 140, 1)
    conv1_w = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 3], stddev=0.1, seed=54), name="conv1_w")
    conv1_b = tf.Variable(tf.zeros(shape=[3]), name="conv1_b")
    
    # (N, 28, 140, 3)
    
    # (N, 10, 47, 3) -> (N, 1410)
    dens3_w = tf.Variable(tf.truncated_normal(shape=[1410, 200], stddev=0.1, seed=65), name="dens3_w")
    dens3_b = tf.Variable(tf.zeros(shape=[200]), name="dens3_b")
    
    # (N, 200)
    len_w = tf.Variable(tf.truncated_normal(shape=[200, 5], stddev=0.1, seed=76), name="len_w")
    len_b = tf.Variable(tf.zeros(shape=[5]), name="len_b")
    
    # (N, 200)
    dig_w = tf.Variable(tf.truncated_normal(shape=[200, 5*10], stddev=0.1, seed=87), name="dig_w")
    dig_b = tf.Variable(tf.zeros(shape=[5*10]), name="dig_b")

In [280]:
def model(x):
    o1 = tf.nn.relu(tf.nn.conv2d(x, conv1_w, [1, 1, 1, 1], 'SAME') + conv1_b)
    o2 = tf.nn.max_pool(o1, [1, 3, 3, 1], [1, 3, 3, 1], 'SAME')
    o2resh = tf.reshape(o2, [-1, 1410])
    o3 = tf.nn.relu(tf.matmul(o2resh, dens3_w) + dens3_b)
    return o3

In [292]:
with simplest_graph.as_default():
    
    hrep = model(tf_x)
    tf_lenlogit = tf.matmul(hrep, len_w) + len_b
    tf_lenproba = tf.nn.softmax(tf_lenlogit)
    tf_diglogit = tf.reshape(tf.matmul(hrep, dig_w) + dig_b, [-1, 5, 10])
    tf_digproba = tf.nn.softmax(tf_diglogit, dim=-1)
    
    tf_len_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_ylen, logits=tf_lenlogit)
    tf_dig_loss = masked_xe(tf_digproba, tf_y)
    
    optimizer = tf.train.GradientDescentOptimizer(lr)
    updater = optimizer.minimize(tf_len_loss*len_balance + tf_dig_loss*(1-len_balance))

In [295]:
n_epochs = 10
batch_size = 20
steps_per_epoch = 50

In [296]:
session = tf.InteractiveSession(graph=simplest_graph)
init = tf.global_variables_initializer()
init.run()
np.random.seed(123456)
for ep in range(n_epochs):
    idx = np.arange(len(train_x))
    np.random.shuffle(idx) # shuffling the data each epoch
    train_x = train_x[idx]
    train_y = train_y[idx]
    train_ylen = train_ylen[idx]
    ep_len_losses = []
    ep_dig_losses = []
    for batch_idx in range(min(len(train_x)//batch_size, steps_per_epoch)):
        batch_x = train_x[batch_size*batch_idx:(batch_size*(batch_idx+1))].reshape(-1, 28, 140, 1)
        batch_y = train_y[batch_size*batch_idx:(batch_size*(batch_idx+1))]
        batch_ylen = train_ylen[batch_size*batch_idx:(batch_size*(batch_idx+1))]
        _,  batch_len_loss, batch_dig_loss = session.run([updater, tf_len_loss, tf_dig_loss], 
                                                         feed_dict={tf_x: batch_x, tf_y: batch_y, tf_ylen: batch_ylen})
        ep_len_losses.append(np.mean(batch_len_loss))
        ep_dig_losses.append(np.mean(batch_dig_loss))
        
    v_len_loss, v_dig_loss, v_len_proba, v_dig_proba = session.run([tf_len_loss, tf_dig_loss, tf_lenproba, tf_digproba], 
                                                                {tf_x: valid_x, tf_y: valid_y, tf_ylen: valid_ylen})
#     print(np.array(ep_len_losses))
#     print(np.mean(v_len_loss))
    print("Epoch {}".format(ep))
    print("Validation accuracy {:<2.3%}".format(accuracy(valid_y, valid_ylen, v_dig_proba, v_len_proba)))
          
    print("Mean batch len loss {:<2.3f}".format(np.mean(ep_len_losses)))
    print("Valid len loss {:<2.3f}".format(np.mean(v_len_loss)))
    
    print("Mean batch dig loss {:<2.3f}".format(np.mean(ep_dig_losses)))
    print("Valid dig loss {:<2.3f}".format(np.mean(v_dig_loss)))    

Epoch 0
Validation accuracy 1.650%
Mean batch len loss nan
Valid len loss nan
Mean batch dig loss nan
Valid dig loss nan


KeyboardInterrupt: 

In [ ]:

with simplest_graph.as_default():
    tf_x = tf.placeholder(tf.float32, (None, all_x.shape[1], all_x.shape[2]))
    tf_y = tf.placeholder(tf.float32, (None, all_y.shape[1], all_y.shape[2]))
    tf_ylen = tf.placeholder(tf.int32, (None, 5))
    
#     weights
    def model(x):
        o1 = tf.nn.relu(tf.nn.conv2d(x, conv1_w, [1, 1, 1, 1], 'same') + conv1_b)
        o2 = tf.nn.max_pool(o1, [1, 3, 3, 1], [1, 1, 1, 1], 'same')
        o2resh = tf.reshape(o2, [x.get_shape()[0], -1])
        o3 = tf.nn.relu(tf.matmul(o2resh, dens3_w) + dens3_b)
        return o3
    

    
    
#     tf.train.AdamOptimizer.minimize()
    
    

In [20]:
x = tf.placeholder(tf.float32, (None, 10))
w = tf.Variable(np.arange(10).reshape((10, 1)), dtype=tf.float32)
b = tf.Variable(np.zeros(1), dtype=tf.float32)
y = tf.matmul(x, w) + b

In [23]:
sess = tf.InteractiveSession()

In [24]:
tf.global_variables_initializer().run()
print(y.eval({x: np.arange(10*20).reshape(-1, 10)}))

[[  285.]
 [  735.]
 [ 1185.]
 [ 1635.]
 [ 2085.]
 [ 2535.]
 [ 2985.]
 [ 3435.]
 [ 3885.]
 [ 4335.]
 [ 4785.]
 [ 5235.]
 [ 5685.]
 [ 6135.]
 [ 6585.]
 [ 7035.]
 [ 7485.]
 [ 7935.]
 [ 8385.]
 [ 8835.]]
